In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn
import pickle
from sklearn.cross_validation import KFold

In [ ]:
f = open("C:\\Users\\谢婷谢婷\\Desktop\\creditcard\\train_test.data", 'rb')
# load data
dataset = pickle.load(f)
#generating variables
X_test1=dataset[0]
X_test2=dataset[1]
X_train1=dataset[2]
X_train2=dataset[3]
y_test1=dataset[4]
y_test2=dataset[5]
y_train1=dataset[6]
y_train2=dataset[7]

In [ ]:
# In[3]:
# define plot confusion matrix function

from sklearn.metrics import confusion_matrix,precision_recall_curve,auc,roc_auc_score,roc_curve,recall_score,classification_report 
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")
    else:
        1#print('Confusion matrix, without normalization')

    #print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
# In[4]:
# Logistic Regression
    
# cross validation to choose parameters in Logistic Regression
def printing_Kfold_scores(x_train_data,y_train_data):
    fold = KFold(len(y_train_data),5,shuffle=False) 

    # Different C parameters
    c_param_range = [0.01,0.1,1,10,100]

    results_table = pd.DataFrame(index = range(len(c_param_range),2), columns = ['C_parameter','Mean recall score'])
    results_table['C_parameter'] = c_param_range

    # the k-fold will give 2 lists: train_indices = indices[0], test_indices = indices[1]
    j = 0
    for c_param in c_param_range:
        print('-------------------------------------------')
        print('C parameter: ', c_param)
        print('-------------------------------------------')
        print('')

        recall_accs = []
        for iteration, indices in enumerate(fold,start=1):

            # Call the logistic regression model with a certain C parameter
            lr = LogisticRegression(C = c_param, penalty = 'l1')

            # Use the training data to fit the model. In this case, we use the portion of the fold to train the model
            # with indices[0]. We then predict on the portion assigned as the 'test cross validation' with indices[1]
            lr.fit(x_train_data.iloc[indices[0],:],y_train_data.iloc[indices[0],:].values.ravel())

            # Predict values using the test indices in the training data
            y_pred_undersample = lr.predict(x_train_data.iloc[indices[1],:].values)

            # Calculate the recall score and append it to a list for recall scores representing the current c_parameter
            recall_acc = recall_score(y_train_data.iloc[indices[1],:].values,y_pred_undersample)
            recall_accs.append(recall_acc)
            print('Iteration ', iteration,': recall score = ', recall_acc)

        # The mean value of those recall scores is the metric we want to save and get hold of.
        results_table.ix[j,'Mean recall score'] = np.mean(recall_accs)
        j += 1
        print('')
        print('Mean recall score ', np.mean(recall_accs))
        print('')

    best_c = results_table.loc[results_table['Mean recall score'].idxmax()]['C_parameter']
    
    # Finally, we can check which C parameter is the best amongst the chosen.
    print('*********************************************************************************')
    print('Best model to choose from cross validation is with C parameter = ', best_c)
    print('*********************************************************************************')
    
    return best_c

best_c = printing_Kfold_scores(X_train1,y_train1)

# In[]

##########   Finishing choosing parameters
from sklearn.linear_model import LogisticRegression
# Use this C_parameter to build the final model with the whole training dataset and predict the classes in the test
lr = LogisticRegression(C = 0.01, penalty = 'l1')
lr.fit(X_train1,y_train1.values.ravel())
y_pred1 = lr.predict(X_test1.values)

lr.score(X_test1, y_test1)
print('accuracy of training set1: {:.4f}'.format(lr.score(X_train1,y_train1)))
print('accuaracy of test set1: {:.4f}'.format(lr.score(X_test1, y_test1)))
#accuracy calculation

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test1,y_pred1)
np.set_printoptions(precision=2)

print("Recall metric in the testing dataset: ", cnf_matrix[1,1]/(cnf_matrix[1,0]+cnf_matrix[1,1]))

# Plot non-normalized confusion matrix
class_names = [0,1]
plt.figure()
plot_confusion_matrix(cnf_matrix
                      , classes=class_names
                      , title='Confusion matrix')
plt.show()

# ROC CURVE

y_pred1_score = lr.fit(X_train1,y_train1.values.ravel()).decision_function(X_test1.values)

fpr, tpr, thresholds = roc_curve(y_test1.values.ravel(),y_pred1_score)
roc_auc = auc(fpr,tpr)

# Plot ROC
plt.title('Receiver Operating Characteristic_undersampling')
plt.plot(fpr, tpr, 'b',label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.0])
plt.ylim([-0.1,1.01])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

##  Undersampling finished

lr = LogisticRegression(C = 0.01, penalty = 'l1')
lr.fit(X_train2,y_train2.values.ravel())
y_pred2 = lr.predict(X_test2.values)

lr.score(X_test2, y_test2)
print('accuracy of training set2: {:.4f}'.format(lr.score(X_train2,y_train2)))
print('accuaracy of test set2: {:.4f}'.format(lr.score(X_test2, y_test2)))
#accuracy calculation

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test2,y_pred2)
np.set_printoptions(precision=2)

print("Recall metric in the testing dataset: ", cnf_matrix[1,1]/(cnf_matrix[1,0]+cnf_matrix[1,1]))

# Plot non-normalized confusion matrix
class_names = [0,1]
plt.figure()
plot_confusion_matrix(cnf_matrix
                      , classes=class_names
                      , title='Confusion matrix')
plt.show()

# ROC CURVE
y_pred2_score = lr.fit(X_train2,y_train2.values.ravel()).decision_function(X_test2.values)

fpr, tpr, thresholds = roc_curve(y_test2.values.ravel(),y_pred2_score)
roc_auc = auc(fpr,tpr)

# Plot ROC
plt.title('Receiver Operating Characteristic_Oversampling')
plt.plot(fpr, tpr, 'b',label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.0])
plt.ylim([-0.1,1.01])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show

In [ ]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
knc = KNeighborsClassifier(n_neighbors = 3)
knc.fit(X_train1,y_train1)
y_knc = knc.predict(X_test1)
print('accuracy of training set1: {:.4f}'.format(knc.score(X_train1,y_train1)))
print('accuracy of test set1: {:.4f}'.format(knc.score(X_test1, y_test1)))
#when choosing n_neighbors=3,we have the maximum acuracy of test set1

knc = KNeighborsClassifier(n_neighbors = 3)
knc.fit(X_train2,y_train2)
y_knc = knc.predict(X_test2)
print('accuracy of training set2: {:.4f}'.format(knc.score(X_train2,y_train2)))
print('accuracy of test set2: {:.4f}'.format(knc.score(X_test2, y_test2)))

In [ ]:
#SVM
# Scikit-learn library: For SVM
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn import svm
import itertools

classifier = svm.SVC(kernel='linear') 
classifier.fit(X_train1, y_train1)
prediction_SVM_all1 = classifier.predict(X_test1)
cm1 = confusion_matrix(y_test1, prediction_SVM_all1)
print('We have detected ' + str(cm1[1][1]) + ' frauds / ' + str(cm1[1][1]+cm1[1][0]) + ' total frauds.')
print('\nSo, the probability to detect a fraud is ' + str(cm1[1][1]/(cm1[1][1]+cm1[1][0])))
print("the accuracy is : "+str((cm1[0][0]+cm1[1][1]) / (sum(cm1[0]) + sum(cm1[1]))))
#So, the probability to detect a fraud is 0.89 the accuracy is : 0.9390862944162437


classifier = svm.SVC(kernel='linear') 
classifier.fit(X_train2, y_train2)
prediction_SVM_all2 = classifier.predict(X_test2)
cm2 = confusion_matrix(y_test2, prediction_SVM_all2)
print('We have detected ' + str(cm2[1][1]) + ' frauds / ' + str(cm2[1][1]+cm1[1][0]) + ' total frauds.')
print('\nSo, the probability to detect a fraud is ' + str(cm2[1][1]/(cm1[1][1]+cm2[1][0])))
print("the accuracy is : "+str((cm2[0][0]+cm2[1][1]) / (sum(cm2[0]) + sum(cm2[1]))))